In [7]:
import psycopg2
import datetime
import json
from sqlalchemy import create_engine, text
import pandas as pd

In [8]:
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records

In [9]:
def pretty_select(query):
    connection_string = 'postgres://{}:{}@{}:5432/{}'.format('okgedrkq', 'OWJG1VssXGsfuUWTguGQoCmtZ5c5JZIp', 'lallah.db.elephantsql.com', 'okgedrkq')
    try:
        engine = create_engine(connection_string)
        records = pd.read_sql_query(query, engine)
    except Exception as e:
        print('Error en el query:', e)
    else:
        return records

In [10]:
myConnection = psycopg2.connect(host = 'lallah.db.elephantsql.com',
                                user= 'okgedrkq', password ='OWJG1VssXGsfuUWTguGQoCmtZ5c5JZIp',
                                dbname= 'okgedrkq')

## Función de compras

In [ ]:
def carrito():
    query = """CREATE TYPE carrito AS(producto INTEGER, cantidad INTEGER)"""
    cur = myConnection.cursor()
    cur.execute(query)
    cur.close()
    myConnection.commit()
carrito()

In [ ]:
def comprar(producto, idcliente, banco, fecha, idsucursal):
    query = """CREATE OR REPLACE FUNCTION compra(producto carrito[], idCliente INTEGER, banco varchar(25), fecha timestamp, idSucursal INTEGER)
    RETURNS void as $$
    DECLARE
        factura integer;
		pago integer;
    BEGIN
        IF ARRAY_LENGTH(producto,1)>0 THEN
        INSERT INTO factura(fecha, is_pagada, monto_total, iva, idCliente, idSucursal, banco)
        VALUES(fecha, true, 100, 12.90, idCliente, idSucursal, banco) RETURNING factura.id INTO factura;
        INSERT INTO pago(monto,id_factura)
        VALUES(0,factura) RETURNING pago.id INTO pago;
        INSERT INTO Ventas(cant, monto, id_prod, id_factura)
        SELECT c.cantidad,0,c.producto,factura FROM UNNEST(producto) AS c;
    ELSE
        RAISE EXCEPTION 'The cart is empty or null %', now();
    END IF;
    END;
    $$ 
    LANGUAGE plpgsql"""
    cur = myConnection.cursor()
    cur.execute(query)
    cur.close()
    myConnection.commit()
    
    query = f""" SELECT compra(array{producto}::carrito[], '{idcliente}', '{banco}', '{fecha}', {idsucursal})"""
    cur = myConnection.cursor()
    cur.execute(query)
    cur.close()
    myConnection.commit()
    
# mock client

client = '{"client": "1", "list":[{"producto":1, "cant":8}, {"producto":2, "cant":17}]}'
data = json.loads(client)
car = []
for i in data['list']:
    car.append((i['producto'], i['cant']))
    
comprar(car, 2, 'Banesco', '2020-11-10', 2)
        
    
    

## Funcion de estado de cuenta

In [ ]:
print(datetime.datetime.now())

In [ ]:
def estadoDeCuenta(diaActual):
    query ="""CREATE OR REPLACE FUNCTION estadoDeCuenta1(dia TIMESTAMP)
    RETURNS TABLE(BANCOACTUAL VARCHAR(25), MONTO FLOAT, fecha DATE) as $$
    DECLARE
        bancos varchar[] := '{"Provincial","Mercantil","Banesco"}';
		montoAux FLOAT;
		i VARCHAR;
    BEGIN
        FOREACH i IN ARRAY bancos LOOP
			BANCOACTUAL:=i;
			IF i IN(SELECT FACTURA.BANCO FROM FACTURA
				   WHERE DATE(FACTURA.FECHA)<= DATE(dia))THEN
				   SELECT SUM(MONTO_TOTAL) INTO montoAux FROM factura
				   WHERE banco=i AND DATE(FACTURA.FECHA) <= DATE(DIA);
				   MONTO:=montoAux;
				   fecha := dia;
				   montoAux := 0;
				   RETURN NEXT;
    		ELSE
				MONTO :=0;
				FECHA:=DIA;
				RETURN NEXT;
			END IF;
		END LOOP;
	END;		
    $$ LANGUAGE plpgsql"""
    cur = myConnection.cursor()
    cur.execute(query)
    cur.close()
    myConnection.commit()
    estadoDeCuenta(datetime.datetime.now())

In [6]:
query = f"""SELECT * FROM estadoDeCuenta1('{datetime.datetime.now()}')"""
pretty_select(text(query))

bancoactual  monto       fecha
0  Provincial    0.0  2020-11-10
1   Mercantil    0.0  2020-11-10
2     Banesco  100.0  2020-11-10

## Funcion puntos de afiliados 

In [12]:
def puntosDeClientes(ultimoDiaMes):
    query = """CREATE OR REPLACE FUNCTION puntosDeAfiliados(ultimoDiaMes timestamp)
	RETURNS TABLE(clienteId INTEGER, puntos INTEGER) AS $$
	DECLARE
		X RECORD;
		ULTIMO DATE;
	BEGIN
	SELECT (date_trunc('MONTH',DATE(ultimoDiaMes)) + INTERVAL '1 month' - INTERVAL '1 day')::date into ultimo;
	IF EXTRACT(DAY FROM ultimoDiaMes) = EXTRACT(DAY FROM ULTIMO) THEN
	FOR X IN (SELECT DISTINCT id_cliente AS idCliente, club_clientes.puntos FROM club_clientes
			 INNER JOIN FACTURA ON idCliente = FACTURA.idCliente
			 WHERE factura.idCliente = idCliente AND FACTURA.fecha > ultimoDiaMes - INTERVAL '1 MONTH' AND 
			 FACTURA.FECHA < ultimoDiaMes + INTERVAL '1 day' ORDER BY puntos DESC) LOOP
			 	clienteId := X.idcliente;
				puntos := X.puntos;
				RETURN NEXT;
			 END LOOP;
	ELSE
		RAISE EXCEPTION 'Debe usar la funcion a final de mes%',EXTRACT(DAY FROM ultimoDiaMes);
	END IF;
 END;
 $$ 
 LANGUAGE plpgsql"""
    cur = myConnection.cursor()
    cur.execute(query)
    cur.close()
    myConnection.commit()
    puntosDeClientes('2020-11-30')

In [14]:
query = f"""SELECT * FROM puntosDeAfiliados('2020-11-30')"""
pretty_select(text(query))

clienteid  puntos
0          2      10
1          1       0